# Imagenet results

+ classification performance metrics
     + AUROC
     + AUPRC
     + Accuracy
     


In [1]:
import os
import pickle
import time
import gc

import torch
import torch.nn as nn
import torchvision.models as models

import torchmetrics
from torchmetrics.classification import BinaryAUROC
from torchmetrics.classification import BinaryAveragePrecision
from torchmetrics.classification import BinaryAccuracy

from torchmetrics.classification import MulticlassAccuracy
from torchmetrics.classification import MulticlassAveragePrecision
from torchmetrics.classification import MulticlassAUROC


In [2]:
import torchmetrics
torchmetrics.__version__


'0.11.4'

In [3]:
import os
import nibabel as nib

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle
import random

from scipy import ndimage

from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage import transform
from skimage.transform import resize
import skimage.exposure as skie

import ot

import torch
from torch import manual_seed
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torchvision.transforms as transforms
from torchsummary import summary
import torchvision
from torchvision import datasets, models, transforms

from torchmetrics import Accuracy

import numpy as np
import matplotlib.pyplot as plt
import time
import os
import copy

from PIL import Image 

torch.cuda.is_available()

torch.set_num_threads(3)

def compactness(blob_labels):
    import math
    compactness=[]
    region=regionprops(blob_labels)
    for rp in region:
        area=rp.area
        #perimeter=rp.perimeter
        perimeter=rp.perimeter_crofton
        c=(4*math.pi*area)/(perimeter**2)
        compactness.append(c)
    return compactness

def get_blobs(image,lesion_size):
    """
    gets the noise image after pre processing and returns blobs with the size equal to lesion size
    image is the noise image 
    lesion_size can be an int
    """
    
    labeled, nr_objects = ndimage.label(image)
    sizes = ndimage.sum_labels(image,labeled,range(nr_objects+1))    
    mask_size=sizes!=lesion_size
    small_blobs=labeled.copy()
    remove_pixel = mask_size[small_blobs]
    small_blobs[remove_pixel] = 0 
    
    return small_blobs


def list_lesions(image,lesion_size):
    round_lesion_c=0.8 #lesions with compactness above this value are considered round
    not_round_lesion_c=0.4 #lesions with compactness bellow this value are considered not round
    
    border=2
    blur=0.75
    
    less_size=lesion_size*0.05*blur #the lesion is smaller after the blur
    
    small_blobs=get_blobs(image,lesion_size)
    rb=regionprops(small_blobs)
    round_lesions=[]
    not_round_lesions=[]
    c_list=compactness(small_blobs)
    
    for blob in range(len(c_list)):
        
        if c_list[blob]>round_lesion_c:#round lesions

            blob_img=rb[blob].image.astype(float)
            #the image is padded because when it is smoothed it increases a bit
            pad_img=np.pad(array=blob_img, pad_width=border, mode='constant', constant_values=0)
            blur_image=ndimage.gaussian_filter(pad_img, blur)
            blur_image[blur_image<0.2]=0
            energy=round(np.sum(blur_image.astype(float)),2)
            if energy<=lesion_size-less_size+1 and energy>=lesion_size-less_size-1:
                round_lesions.append([blur_image,round(c_list[blob],3)])
        
        
        elif c_list[blob]<not_round_lesion_c:# not round lesions
            blob_img=rb[blob].image.astype(float)
            #the image is padded because when it is smoothed it increases a bit
            pad_img=np.pad(array=blob_img, pad_width=border, mode='constant', constant_values=0)
            blur_image=ndimage.gaussian_filter(pad_img, blur)
            blur_image[blur_image<0.15]=0
            energy=round(np.sum(blur_image.astype(float)),2)
            if energy<=lesion_size-less_size+1 and energy>=lesion_size-less_size-1:
                not_round_lesions.append([blur_image,round(c_list[blob],3)])
            
    return round_lesions, not_round_lesions

def create_lesions(lesion_number,lesion_size,factor=1):
    #factor is the value for which we multiply the sides of the lesion
    round_lesions=[]
    not_round_lesions=[]
    s=0
    size_noise=256
    blur_radius=2

    while len(not_round_lesions)<=lesion_number or len(round_lesions)<=lesion_number:
        #create noise
        np.random.seed(s)
        noise_img=np.random.rand(size_noise,size_noise)

        #smooth noise
        imgf=ndimage.gaussian_filter(noise_img, blur_radius)

        #create binary image
        thr=threshold_otsu(imgf)
        imgf_bin=imgf>thr
        
        #morphologic changes
        erosion_image=ndimage.binary_erosion(imgf_bin)
        open_er_img=ndimage.binary_opening(erosion_image)
        erosion_image2=ndimage.binary_erosion(open_er_img)

        #images
        

        #as a result from one noise image we create several images that can be used to create the lesions
        round_lesions_open, not_round_lesions_open=list_lesions(open_er_img,lesion_size)
        round_lesions_er, not_round_lesions_er=list_lesions(erosion_image2,lesion_size)
        
        round_lesions=round_lesions+round_lesions_open+round_lesions_er
        not_round_lesions=not_round_lesions+not_round_lesions_open+not_round_lesions_er
        
        
        #print('len lists:',len(round_lesions),len(not_round_lesions))
        '''        
        plt.figure(figsize=(15,7))
        plt.subplot(2,4,1)
        plt.imshow(noise_img)
        plt.title('noise')
        plt.subplot(2,4,2)
        plt.imshow(imgf)
        plt.title('blur')
        plt.subplot(2,4,3)
        plt.imshow(imgf_bin)
        plt.title('binary img')
        plt.subplot(2,4,4)
        plt.imshow(erosion_image)
        plt.title('erosion')
        plt.subplot(2,4,5)
        plt.imshow(open_er_img)
        plt.title('open')
        plt.subplot(2,4,6)
        plt.imshow(erosion_image2)
        plt.title('erosion2')
        plt.subplot(2,4,7)
        plt.imshow(dilated_image)
        plt.title('dilation')
        break
        '''
        
        s+=1
    print(f'round lesions: {len(round_lesions)}/{lesion_number} === not round lesions {len(not_round_lesions)}/{lesion_number}')
    print(f'number of seeds used: {s-1}')
    print(f'the lesions have size between {lesion_size-lesion_size*0.05*0.75-1} and {lesion_size-lesion_size*0.05*0.75+1}')
    
    lesions_r=round_lesions[:lesion_number]
    lesions_nr=not_round_lesions[:lesion_number]
    round_lesions=[[np.array(resize(round_lesions[i][0],(round_lesions[i][0].shape[0]*factor,round_lesions[i][0].shape[1]*factor))),round_lesions[i][1]] for i in range(len(lesions_r))]
    not_round_lesions=[[np.array(resize(not_round_lesions[i][0],(not_round_lesions[i][0].shape[0]*factor,not_round_lesions[i][0].shape[1]*factor))),not_round_lesions[i][1]] for i in range(len(lesions_nr))]
    
    return round_lesions, not_round_lesions

def rescale_values(image,max_val,min_val):
    '''
    image - numpy array
    max_val/min_val - float
    '''
    return (image-image.min())/(image.max()-image.min())*(max_val-min_val)+min_val

def select_coordinates(slice_image, lesions,white_constant,seed):
    '''
    slice_image is the brain slice to use
    lesions is a list of the lesions (with len=number_of_lesions) to use
    colour_lesion is either 'black' or 'white'
    white_constant is the constant that is multiplied with the lesion mask to create lighter or darker lesions
    '''

    np.random.seed(seed)
    
    brain_mask=np.array(slice_image)>0
    brain_mask=brain_mask.astype(float)
    x,y = np.where(brain_mask == 1.)
    
    lesion_brain=slice_image.copy().astype(float)
    lesion_mask=brain_mask.copy() 
    lesion_added=0
    ground_truth=np.zeros(slice_image.shape)
    min_value=0.1 #min value for the lesion intensity 
    max_value=0.9 #max value for the lesion intensity
    brain_image=slice_image.copy()
    
    while lesion_added<len(lesions):
        i=np.random.choice(np.arange(len(x)))
        coordinate=[x[i],y[i]]
        lesion=lesions[lesion_added]
        lesion_rescale = rescale_values(lesion,max_value,min_value)
        lesion_rescale=rescale_values(lesion,white_constant,min_value)
        
        #creating the lesion mask and ground truth
        if (brain_mask[coordinate[0]: coordinate[0] + lesion.shape[0], coordinate[1]: coordinate[1] + lesion.shape[1]] ==1).all():
            #checks if the lesion that will be added is completly in a white space of the lesion mask 
            #(this means that the new lesion is not overlaping an existing one and is completly in the brain area)
            lesion_mask[coordinate[0]: coordinate[0] + lesion.shape[0], coordinate[1]: coordinate[1] + lesion.shape[1]] -= lesion_rescale
            lesion_added+=1
            ground_truth[coordinate[0]: coordinate[0] + lesion.shape[0], coordinate[1]: coordinate[1] + lesion.shape[1]]+=lesion
            brain_mask=lesion_mask

    
    brain_image=slice_image.copy()
    brain_mask=np.array(slice_image)>0
    #creating the white lesions
       
    brain_image[brain_image>0]=1-brain_image[brain_image>0]        
    brain_image[lesion_mask!=0]*=lesion_mask[lesion_mask!=0]
    brain_image[brain_mask]=1-brain_image[brain_mask]
    

    
    return lesion_mask,brain_image,ground_truth
    
def add_lesions(slice_image,round_lesions,not_round_lesions,min_lesion,max_lesion,white_constant,seed,max_brain):
    #for each slice we chose: random number of lesions, random lesions, random coordinates
    
    np.random.seed(seed)
    number_of_lesions=np.random.randint(min_lesion,max_lesion+1)
    lesion_type=np.random.randint(0,2)
    #lesion_type=0 - round
    #lesion_type=1 - not round
    
    #get lesions from type of lesions (and target)
    added=rescale_values(slice_image.copy(),max_brain,0)
    if lesion_type==0: #round
        target=0
        with open('round_lesions.pkl', 'rb') as f:
            lesion_list=pickle.load(f)
        np.random.shuffle(round_lesions)
        
    elif lesion_type==1: #not round
        target=1
        with open('not_round_lesions.pkl', 'rb') as f:
            lesion_list=pickle.load(f) 
        np.random.shuffle(lesion_list)
        
    lesions=[i[0] for i in lesion_list[:number_of_lesions]]
    #add the lesions
    
    lesion_mask,lesion_brain_white,ground_truth=select_coordinates(added, lesions,white_constant,seed)
    
    
    return lesion_mask,lesion_brain_white,ground_truth,target,number_of_lesions

def change_images(image):
    image=np.repeat(image[..., np.newaxis], 3, axis=2)
    image=resize(image, (224, 224))
    image=image.transpose(2,0,1)
    return image

def create_dataset(slices,round_lesions,not_round_lesions,min_lesion=3,max_lesion=5,white_constant=0.85,seed=0,max_brain=1):

    dataset_white=[]
    number_lesions=[]
    lesion_mask_list=[]
    ground_truths=[]
    for slice_idx in range(len(slices)):
        lesion_mask,lesion_brain_white,ground_truth,target,number_of_lesions=add_lesions(slices[slice_idx],
                                                                                         round_lesions,
                                                                                         not_round_lesions,
                                                                                         min_lesion=min_lesion,
                                                                                         max_lesion=max_lesion,
                                                                                         white_constant=white_constant,
                                                                                        seed=seed,
                                                                                        max_brain=max_brain)
        dataset_white.append([change_images(lesion_brain_white),target])
        number_lesions.append(number_of_lesions)
        lesion_mask_list.append(lesion_mask)
        ground_truths.append(ground_truth)
        seed+=1
        
        if slice_idx%1500==0:
            print(f'slice {slice_idx}/{len(slices)} = {round(100*slice_idx/len(slices),2)}%')
        
    return dataset_white,number_lesions,lesion_mask_list,ground_truths
        
    


In [4]:
def load_VGG_model(path,device):
    model = models.vgg16(pretrained=True)
    model.classifier=model.classifier[:-1]
    last_layers=[nn.Linear(4096,2)]
    model.classifier = nn.Sequential(*list(model.classifier)+last_layers) 

    model.load_state_dict(torch.load(path,map_location=device))

    return model



### Best performing models

In [6]:
def obtain_labels(model, dataset, DEVICE):
    BATCH = 32
    dataloader = DataLoader(dataset,batch_size=BATCH)

    real_labels = []
    pred_labels = []
    logits_list = []
    
    model = model.to(DEVICE)
    
    for inputs, labels in dataloader:

        inputs = inputs.to(DEVICE,dtype=torch.float)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(DEVICE)

        logits = model(inputs)
        
        real_labels += torch.Tensor.cpu(labels).tolist()
        
        logits_list += torch.Tensor.cpu(logits).tolist()
        
        pred_labels += torch.Tensor.cpu(torch.max(logits, 1)[1]).tolist()
        
    model=None
    inputs=None
    labels=None
    logits=None
    gc.collect()
    torch.cuda.empty_cache()

    return real_labels, pred_labels, logits_list


In [59]:
 model=None
inputs=None
labels=None
logits=None
gc.collect()
torch.cuda.empty_cache()

In [7]:
def classification_performance(logits, target):
#     metric = BinaryAUROC(thresholds=None)
#     AUROC = metric(logits, target)
    
#     metric = BinaryAveragePrecision(thresholds=None)
#     AUPRC = metric(logits, target) # preds are logits  
    
#     metric = BinaryAccuracy()
#     ACC = metric(logits, target)

    metric = MulticlassAveragePrecision(num_classes=2)
    AUPRC = metric(logits, target)

    metric = MulticlassAccuracy(num_classes=2)
    ACC = metric(logits, target)

    metric = MulticlassAUROC(num_classes=2)
    AUROC = metric(logits, target)

    return AUROC, AUPRC, ACC 


In [10]:
# creating dataset

seed=0
np.random.seed(seed)
random.seed(seed)
plt.rc('image',cmap='gray')  

# creating lesions
number_of_lesions=50 #amount of lesions in each lesion list
size_of_lesions=70 #size of all the lesions
factor=2

round_lesions, not_round_lesions=create_lesions(number_of_lesions,size_of_lesions,factor=factor)

done = time.time()
elapsed = done - start

# loading slices 
with open('slices_validation.pkl', 'rb') as f:
    validation_slices,target_valid_gender,target_valid_age = pickle.load(f)
    
    
# adding lesions to slices
lesion_max_intensity=0.5
max_brain_intensity=0.7

start = time.time()
print(' ====== holdout ====== ')

dataset,_,_,ground_truths=create_dataset(validation_slices,
                                          round_lesions,
                                          not_round_lesions,
                                          min_lesion=3,
                                          max_lesion=5,
                                          white_constant=lesion_max_intensity,
                                          seed=50000,
                                          max_brain=max_brain_intensity)

target_w=[i[1] for i in dataset]
print(f'{len([i for i in target_w if i==1])} slices of target 1 out of {len(target_w)} slices: {round(100*len([i for i in target_w if i==1])/len(target_w),2)} %')
print(f' number of slices: {len(dataset)}')


round lesions: 51/50 === not round lesions 207/50
number of seeds used: 886
the lesions have size between 66.375 and 68.375
took 10.5s
 ====== holdout ====== 
slice 0/8539 = 0.0%
slice 1500/8539 = 17.57%
slice 3000/8539 = 35.13%
slice 4500/8539 = 52.7%
slice 6000/8539 = 70.27%
slice 7500/8539 = 87.83%

took 153.08s
4277 slices of target 1 out of 8539 slices: 50.09 %
 number of slices: 8539


NameError: name 't0' is not defined

In [122]:
def save_class_metrics(folder, models_names, dataset, output_file_path, device = 'cuda:0'):
    # saves the real and predicted labels, the logits and the three classification metrics
    # based on each of the models in model_names
    #
    # folder is the path to the folder where the models are saved
    # model_names is a list of strings that correspond to the name of the file of the model
    # output_file_name is the string that corresponds to the folder where the list of dictionries
    # will be saved
    #
    # returns to_save_img, a list of the dictionaries obtained from each model in model_names
    
    to_save = []
    n=1
    for model_path in models_names:

        print(f'model number: {n} out of {len(models_names)} \n {model_path}')
        print('load_model...')
        model = load_VGG_model(folder + model_path, device)
        real_labels, pred_labels, logits = obtain_labels(model, dataset, device)

        logits = torch.FloatTensor(logits)
        real_labels = torch.FloatTensor(real_labels).to(torch.int64)

        print('obtain performance...')
        AUROC, AUPRC, ACC = classification_performance(logits, real_labels)

        dics_img = {}
        dics_img['model'] = model_path[:-3]
        dics_img['AUROC'] = AUROC
        dics_img['ACC'] = ACC
        dics_img['AUPRC'] = AUPRC
        dics_img['real_labels'] = real_labels
        dics_img['pred_labels'] = pred_labels
        dics_img['logits'] = logits
        dics_img['block'] = model_path[15]
        #inverts the name, finds the index with '_', selects the chars from that (in the not inverted name: len(name)-index) untill the end 
        dics_img['seed'] = model_path[len(model_path)-model_path[::-1].find('_'):-3]
        
        to_save.append(dics_img)
        
        n+=1
        print()
            
    print()
    print('===================================')
    print('save performances...')
    
    to_save_dump = to_save
    with open(output_file_path+'to_save.pkl', 'wb') as fp:
        pickle.dump(to_save_dump, fp)
       
    return to_save

In [116]:
folder = '/home/martao/MRI_dataset/2ndTry/Models/VGG/2500/best_acc/done/'
models_names = os.listdir(folder)
models_names = [i for i in models_names if (i[0]=='n' and i.find('MRI')==-1)]

device = 'cuda:6'

out_folder = '/home/martao/MRI_dataset/2ndTry/Models/VGG/2500/best_acc/saves/'

best_models_dict_list = save_class_metrics(folder, models_names, dataset, out_folder, device)

model number: 1 out of 25 
 new_finetuning_1conv_0.5_img_2500_0.02_18464.pt
load_model...
obtain performance...
model number: 2 out of 25 
 new_finetuning_all_0.5_img_2500_0.004_58461.pt
load_model...
obtain performance...
model number: 3 out of 25 
 new_finetuning_1conv_0.5_img_2500_0.02_646976.pt
load_model...
obtain performance...
model number: 4 out of 25 
 new_finetuning_2conv_0.5_img_2500_0.008_18464876.pt
load_model...
obtain performance...
model number: 5 out of 25 
 new_finetuning_4conv_0.5_img_2500_0.008_32323548.pt
load_model...
obtain performance...
model number: 6 out of 25 
 new_finetuning_4conv_0.5_img_2500_0.008_116560000.pt
load_model...
obtain performance...
model number: 7 out of 25 
 new_finetuning_4conv_0.5_img_2500_0.008_98794515.pt
load_model...
obtain performance...
model number: 8 out of 25 
 new_finetuning_3conv_0.5_img_2500_0.008_98794515.pt
load_model...
obtain performance...
model number: 9 out of 25 
 new_finetuning_1conv_0.5_img_2500_0.02_23548.pt
load_mo

In [117]:
for name in sorted([n['model'] for n in best_models_dict_list]):
    for i in best_models_dict_list:
        if i['model']==name:
            print(i['model'], '||', i['AUROC'],'||', i['ACC'], '||', i['AUPRC'])

new_finetuning_1conv_0.5_img_2500_0.02_18464 || tensor(0.8280) || tensor(0.7429) || tensor(0.8293)
new_finetuning_1conv_0.5_img_2500_0.02_23548 || tensor(0.8330) || tensor(0.7457) || tensor(0.8342)
new_finetuning_1conv_0.5_img_2500_0.02_55168461 || tensor(0.8288) || tensor(0.7444) || tensor(0.8310)
new_finetuning_1conv_0.5_img_2500_0.02_646976 || tensor(0.8354) || tensor(0.7423) || tensor(0.8373)
new_finetuning_1conv_0.5_img_2500_0.02_98794515 || tensor(0.8100) || tensor(0.7249) || tensor(0.8124)
new_finetuning_2conv_0.5_img_2500_0.008_116560000 || tensor(0.9934) || tensor(0.9586) || tensor(0.9936)
new_finetuning_2conv_0.5_img_2500_0.008_18464876 || tensor(0.9944) || tensor(0.9625) || tensor(0.9945)
new_finetuning_2conv_0.5_img_2500_0.008_23548 || tensor(0.9931) || tensor(0.9545) || tensor(0.9933)
new_finetuning_2conv_0.5_img_2500_0.008_32323548 || tensor(0.9939) || tensor(0.9520) || tensor(0.9941)
new_finetuning_2conv_0.5_img_2500_0.008_55168461 || tensor(0.9927) || tensor(0.9576) || 

In [118]:
for b in sorted(set([n['block'] for n in best_models_dict_list])):
    AUROC_mean = 0
    ACC_mean = 0
    AUPRC_mean = 0 
    
    for i in best_models_dict_list:
        if i['block']==b:
            AUROC_mean += i['AUROC']
            ACC_mean += i['ACC']
            AUPRC_mean += i['AUPRC']
    
    print(f'Average classification performance based on the 5 models')
    print(f'AUROC: {AUROC_mean/5} || ACC: {ACC_mean/5} || AUPRC: {AUPRC_mean/5}' )
    print()

Average classification performance based on the 5 models
AUROC: 0.8270662426948547 || ACC: 0.7400565147399902 || AUPRC: 0.8288397789001465

Average classification performance based on the 5 models
AUROC: 0.9935058355331421 || ACC: 0.9570363759994507 || AUPRC: 0.9936791658401489

Average classification performance based on the 5 models
AUROC: 0.9981449842453003 || ACC: 0.9829277992248535 || AUPRC: 0.9981642961502075

Average classification performance based on the 5 models
AUROC: 0.9995126724243164 || ACC: 0.9899336695671082 || AUPRC: 0.9995200037956238

Average classification performance based on the 5 models
AUROC: 0.9995056986808777 || ACC: 0.9907243847846985 || AUPRC: 0.9995131492614746



### Same performing models

In [119]:
folder = '/home/martao/MRI_dataset/2ndTry/Models/VGG/2500/done_same_acc/'
models_names = os.listdir(folder)
models_names = [i for i in models_names if (i[0]=='n' and i.find('MRI')==-1)]

device = 'cuda:3'

out_folder = '/home/martao/MRI_dataset/2ndTry/Models/VGG/2500/done_same_acc/saves/'

to_save_same_img = best_models_dict_list = save_class_metrics(folder, models_names, dataset, out_folder, device)
    


model number: 1 out of 25 
 new_finetuning_all_0.5_img_2500_0.0006_32323548.pt
load_model...
obtain performance...
model number: 2 out of 25 
 new_finetuning_2conv_0.5_img_2500_0.0018_2147893.pt
load_model...
obtain performance...
model number: 3 out of 25 
 new_finetuning_3conv_0.5_img_2500_0.0008_18464876.pt
load_model...
obtain performance...
model number: 4 out of 25 
 new_finetuning_4conv_0.5_img_2500_0.0006_23548.pt
load_model...
obtain performance...
model number: 5 out of 25 
 new_finetuning_2conv_0.5_img_2500_0.0018_13246.pt
load_model...
obtain performance...
model number: 6 out of 25 
 new_finetuning_all_0.5_img_2500_0.0006_55168461.pt
load_model...
obtain performance...
model number: 7 out of 25 
 new_finetuning_all_0.5_img_2500_0.0006_23548.pt
load_model...
obtain performance...
model number: 8 out of 25 
 new_finetuning_2conv_0.5_img_2500_0.0018_116560000.pt
load_model...
obtain performance...
model number: 9 out of 25 
 new_finetuning_all_0.5_img_2500_0.0006_116560000.pt

In [120]:

for name in sorted([n['model'] for n in to_save_same_img]):
    for i in to_save_img_same:
        if i['model']==name:
            print(i['model'], '||', i['AUROC'],'||', i['ACC'], '||', i['AUPRC'])

new_finetuning_1conv_0.5_img_2500_0.02_18464 || tensor(0.8283) || tensor(0.7440) || tensor(0.8290)
new_finetuning_1conv_0.5_img_2500_0.02_23548 || tensor(0.8316) || tensor(0.7456) || tensor(0.8323)
new_finetuning_1conv_0.5_img_2500_0.02_55168461 || tensor(0.8277) || tensor(0.7434) || tensor(0.8296)
new_finetuning_1conv_0.5_img_2500_0.02_646976 || tensor(0.8360) || tensor(0.7425) || tensor(0.8376)
new_finetuning_1conv_0.5_img_2500_0.02_98794515 || tensor(0.8103) || tensor(0.7230) || tensor(0.8132)
new_finetuning_2conv_0.5_img_2500_0.0018_116560000 || tensor(0.9576) || tensor(0.8827) || tensor(0.9590)
new_finetuning_2conv_0.5_img_2500_0.0018_13246 || tensor(0.9585) || tensor(0.8834) || tensor(0.9601)
new_finetuning_2conv_0.5_img_2500_0.0018_2147893 || tensor(0.9487) || tensor(0.8677) || tensor(0.9505)
new_finetuning_2conv_0.5_img_2500_0.0018_5484646 || tensor(0.9558) || tensor(0.8808) || tensor(0.9574)
new_finetuning_2conv_0.5_img_2500_0.0018_6497 || tensor(0.9542) || tensor(0.8722) || t

In [121]:
for b in sorted(set([n['block'] for n in to_save_same_img])):
    AUROC_mean = 0
    ACC_mean = 0
    AUPRC_mean = 0 
    
    for i in to_save_img_same:
        if i['block']==b:
            AUROC_mean += i['AUROC']
            ACC_mean += i['ACC']
            AUPRC_mean += i['AUPRC']
    
    print(f'Average classification performance based on the 5 models')
    print(f'AUROC: {AUROC_mean/5} || ACC: {ACC_mean/5} || AUPRC: {AUPRC_mean/5}' )
    print()

Average classification performance based on the 5 models
AUROC: 0.8267999887466431 || ACC: 0.7396830916404724 || AUPRC: 0.8283335566520691

Average classification performance based on the 5 models
AUROC: 0.9549658894538879 || ACC: 0.8773702383041382 || AUPRC: 0.9565150141716003

Average classification performance based on the 5 models
AUROC: 0.9570051431655884 || ACC: 0.8842871785163879 || AUPRC: 0.9580898284912109

Average classification performance based on the 5 models
AUROC: 0.9525769352912903 || ACC: 0.8798591494560242 || AUPRC: 0.9534769058227539

Average classification performance based on the 5 models
AUROC: 0.9561964273452759 || ACC: 0.8865715861320496 || AUPRC: 0.9570118188858032

